# **Voice of Customer**

Ampika Thanakaew ID: 6210422049

## **Initialize**

In [19]:
!pip install mlxtend
!pip install --upgrade pythainlp
!pip install pyLDAvis
!pip install sefr_cut

Requirement already up-to-date: pythainlp in /usr/local/lib/python3.7/dist-packages (2.3.1)
     |████████████████████████████████| 8.7MB 4.2MB/s 
     |████████████████████████████████| 327kB 37.0MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85393 sha256=8c27b92dc62a5267592f9785207d335f5334c7284b400bd4a5f715325868c85a
  Stored in directory: /root/.cache/pip/wheels/3a/03/34/77e3ece0bba8b86bfac88a79f923b36d805cad63caeba38842
Successfully built pyahocorasick


In [20]:
import pandas as pd
import numpy as np
import gensim
import pythainlp
from pythainlp.tokenize import sent_tokenize, word_tokenize
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=FutureWarning) 

import re
import seaborn as sns
import matplotlib.pyplot as plt
import sefr_cut
sefr_cut.load_model(engine='tl-deepcut-ws1000')

loading model.....
Success


In [21]:
%cd '/content/drive/MyDrive/BADS 7105 CRM/Voice of Customer/'
%ls

/content/drive/MyDrive/BADS 7105 CRM/Voice of Customer
CustomerReviews.csv


In [22]:
import matplotlib as mpl
mpl.font_manager.fontManager.addfont('/content/drive/MyDrive/BADS 7105 CRM/font/Cordia New Bold.ttf')
mpl.rc('font', family='Cordia New', size=12, weight=200)

**Load Data**

In [23]:
df = pd.read_csv('CustomerReviews.csv')
df.head()

,Review ID,Restaurant_ID,Restaurant,User,Headline,Review,Rating
0,1,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,7b16469831074f7abc7824745ee75212,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,5.0
1,2,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,pakkaramonpondej,รสชาติเหมือนทุกสาขา แต่สาขานี้ บริการดี ที่นั่งดี,นึกถึงชาบูญี่ปุ่นยังไงก็ต้อง คิดถึงโมโม่ พาราไ...,5.0
2,3,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,saanowy,ชาบูพรีเมี่ยมสไตล์ญี่ปุ่น เนื้อดีมากกก,มาทานช่วงนี้ สามารถนั่งโต๊ะเดียวกัน หม้อเดียวก...,4.0
3,4,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ployynp,เนื้อดี ไอติมดี คุ้มค่าดี,ถ้านึกถึงชาบูที่มีเนื้อเน้นๆ ในราคาไม่โหดจนเกิ...,4.0
4,5,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,665a902b335b434ab489f00f2fbb477e,อาหารมีคุณภาพ บริการดีค่ะ,เดินมาหน้าร้านแล้วได้กลิ่นชาบูหอมมาก ๆ ประกอบ...,5.0


**Tokenize Words**

In [24]:
# Word Correction
word_correction_dict = {
    'โมโม่' : 'momo', 'มากก' : 'มาก', 'กกก' : 'ก', 'แวท' : 'vat', 'พิซเซอเรีย' : 'pizzeria', 'พิซซ่า' : 'pizza', 
    'เก้บ' : 'เก็บ', 'ชูชิ' : 'ซูชิ', 'ซึป' : 'ซุป', 'ชาบูชิ' : 'shabushi', 'อารามณ์' : 'อารมณ์', 'ส่งน' : 'ส่วน', 'สไลต์' : 'สไลด์', 'ประยุคก์' : 'ประยุกต์',
    'ไอติม' : 'icecream', 'พาราไดส์' : 'paradise', 'พาราไดซ์' : 'paradise', 'อัพเดท' : 'update', 'นาราย' : 'narai ', 'พรีเมี่ยม' : 'พรีเมียม',
    'บ๋วย' : 'บ๊วย', 'เฟรนไชส์' : 'แฟรนไชส์', 'บุพเฟ่' : 'บุฟเฟ่ต์', 'บุฟเฟ่' : 'บุฟเฟ่ต์', 'บุฟเฟ' : 'บุฟเฟ่ต์', 'บุฟเฟต' : 'บุฟเฟ่ต์', 'รสชาต' : 'รสชาติ', 'รสชาติิ' : 'รสชาติ'
}

# Cleansing
def cleanse_text(text):

    text = text.strip().lower()

    for old,new in word_correction_dict.items():
        text = text.replace(old, new)

    # Remove empty spaces
    text = re.sub(r'\u200b', '', text)
    text = re.sub(r'[\r\n]', '', text)

    # Replace punctuations
    text = re.sub(r'[ๆฯ!#$&%\"\'()*+,-./:;<=>?@\[\]\\^_`{}|~]',' ', text)

    # Replace numbers
    text = re.sub(r'\d',' ', text)
    text = re.sub(r'[0-9]','',text)
    text = re.sub(r'[๐-๙]','',text)

    return text

df['cleansed_text'] = df.apply(lambda x: cleanse_text(x['Headline'] + ' ' + x['Review']), axis=1)

# Sentence and Word Tokenization
df['tokens'] = df['cleansed_text'].apply(lambda x: [ w for st in sent_tokenize(x, engine='whitespace+newline') for w in sefr_cut.tokenize(st,k=100)])

stopwords = list(pythainlp.corpus.thai_stopwords())
removed = [' ', '  ', '\n', 'ร้าน', 'ที่' '(', ')', 'นะคะ', 'เค้า', 'ผม', 'คน']
screening = stopwords + removed

def tokenize_with_space(sentence):
  merged = ''
  words = pythainlp.word_tokenize(str(sentence), engine='newmm')
  for word in words:
    if word not in screening:
      merged = merged + ',' + word
  return merged[1:]

In [25]:
df['review_tokenized'] = df['Review'].apply(lambda x: tokenize_with_space(x))
df.head()

,Review ID,Restaurant_ID,Restaurant,User,Headline,Review,Rating,cleansed_text,tokens,review_tokenized
0,1,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,7b16469831074f7abc7824745ee75212,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,5.0,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,"[[ที่, สำคัญ, ของ, ร้าน, นี้, คือ, บริการ, ดี,...","บริการ,ดีมาก,พนักงาน,น่ารัก,สะอาดสะอ้าน,ใส่ใจ,..."
1,2,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,pakkaramonpondej,รสชาติเหมือนทุกสาขา แต่สาขานี้ บริการดี ที่นั่งดี,นึกถึงชาบูญี่ปุ่นยังไงก็ต้อง คิดถึงโมโม่ พาราไ...,5.0,รสชาติเหมือนทุกสาขา แต่สาขานี้ บริการดี ที่นั่...,"[[รสชาติ, เหมือน, ทุก, สาขา], [แต่, สาขา, นี้]...","นึกถึง,ชาบู,ญี่ปุ่น,คิดถึง,โม,โม่,พาราไดซ์,คุณ..."
2,3,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,saanowy,ชาบูพรีเมี่ยมสไตล์ญี่ปุ่น เนื้อดีมากกก,มาทานช่วงนี้ สามารถนั่งโต๊ะเดียวกัน หม้อเดียวก...,4.0,ชาบูพรีเมียมสไตล์ญี่ปุ่น เนื้อดีมากก มาทานช่วง...,"[[ชาบูพรีเมียม, สไตล์, ญี่ปุ่น], [เนื้อ, ดี, ม...","ทาน,นั่ง,โต๊ะ,หม้อ,โต๊ะ,ยังมี,ฉาก,กั้น,น้ำ,ซุป..."
3,4,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ployynp,เนื้อดี ไอติมดี คุ้มค่าดี,ถ้านึกถึงชาบูที่มีเนื้อเน้นๆ ในราคาไม่โหดจนเกิ...,4.0,เนื้อดี icecreamดี คุ้มค่าดี ถ้านึกถึงชาบูที่ม...,"[[เนื้อ, ดี], [icecream, ดี], [คุ้ม, ค่า, ดี],...","นึกถึง,ชาบู,เนื้อ,ราคา,โหด,เกินไป,นึกถึง,โม,โม..."
4,5,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,665a902b335b434ab489f00f2fbb477e,อาหารมีคุณภาพ บริการดีค่ะ,เดินมาหน้าร้านแล้วได้กลิ่นชาบูหอมมาก ๆ ประกอบ...,5.0,อาหารมีคุณภาพ บริการดีค่ะ เดินมาหน้าร้านแล้วได...,"[[อาหาร, มี, คุณ, ภาพ], [บริการ, ดี, ค่ะ], [เด...","เดิน,หน้า,ได้กลิ่น,ชาบู,หอ,มมาก,โปร,บัตรเครดิต..."


In [26]:
documents = df['review_tokenized'].to_list()
texts = [[text for text in doc.split(',')] for doc in documents]
dictionary = gensim.corpora.Dictionary(texts)

In [27]:
print(dictionary.token2id.keys())

dict_keys(['1', '4', '560', 'กิน', 'ข้างนอก', 'ความต้องการ', 'คอ', 'คุณภาพ', 'คุ้ม', 'จำกัด', 'ซุป', 'ดำ', 'ดี', 'ดีมาก', 'ถาด', 'ถาม', 'น่ารัก', 'น้ำ', 'น้ำจิ้ม', 'บริการ', 'บาท', 'ประทับใจ', 'พนักงาน', 'พอกับ', 'ภาพ', 'รสชาติ', 'ราคา', 'สรุป', 'สะอาดสะอ้าน', 'สัน', 'หลากหลาย', 'อร่อย', 'อันนี้', 'อาหาร', 'เคลม', 'เซ็ท', 'เต้าเจี้ยว', 'เนื้อ', 'เป็ด', 'เผ็ด', 'แบบนี้', 'แปลก', 'แฟน', 'โดยรวม', 'โมจิ', 'ใส่ใจ', 'ไม่ต้อง', 'ไอติม', '\u200b', '           ', '21', '30', '9', 'CTW', 'Terminal', 'กา', 'การงด', 'คิดถึง', 'ชาบู', 'ญี่ปุ่น', 'ฐาน', 'ทะยอย', 'ทำงาน', 'ที่ทาง', 'นาที', 'นึกถึง', 'พนักงานบริการ', 'พระราม', 'พรีเมียม', 'พาราไดซ์', 'มาตรา', 'มีโอ', 'รายละเอียด', 'ลูกค้า', 'สมาทาน', 'สาขา', 'สู้', 'ออร์เดอร์', 'เครื่องดื่ม', 'เจอ', 'เป็นกันเอง', 'เหวี่ยง', 'แจ้ง', 'โม', 'โม่', 'โอเค', '', '++', '2', '555', '600', ':', '::', 'กก', 'กรุบ', 'กั้น', 'ของหวาน', 'คนละ', 'คาว', 'คุ', 'คุ้มค่า', 'จาน', 'จิ้ม', 'จืด', 'ฉาก', 'ชอบ', 'ชาเขียว', 'ชิ้น', 'ซอย', 'ซอส', 'ตก', 'ตะ', 'ตัก', 'ตามน้ำ'

In [28]:
gensim_corpus = [dictionary.doc2bow(text, allow_update=True) for text in texts]
word_frequencies = [[(dictionary[id], frequence) for id, frequence in couple] for couple in gensim_corpus]

**Topic Modeling**

In [29]:
num_topics = 5
chunksize = 100   # size of the doc looked at every pass
passes = 20         # number of passes through documents
iterations = 100    
eval_every = 1     # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = gensim.models.LdaModel(corpus=gensim_corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

CPU times: user 336 ms, sys: 9.73 ms, total: 345 ms
Wall time: 343 ms


In [30]:
pyLDAvis.gensim_models.prepare(model, gensim_corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.061564  0.007573       1        1  34.672519
2      0.006505  0.038525       2        1  19.188024
0      0.039369  0.013443       3        1  18.988380
3     -0.001309 -0.038153       4        1  13.611068
4      0.016999 -0.021388       5        1  13.540010, topic_info=        Term       Freq      Total Category  logprob  loglift
48         ​   5.000000   5.000000  Default  30.0000  30.0000
91         :  11.000000  11.000000  Default  29.0000  29.0000
336        -  10.000000  10.000000  Default  28.0000  28.0000
151    เลือก  18.000000  18.000000  Default  27.0000  27.0000
117      ทาน  23.000000  23.000000  Default  26.0000  26.0000
..       ...        ...        ...      ...      ...      ...
11        ดำ   1.001732   6.873558   Topic5  -5.2508   0.0736
105      ชอบ   1.111561   8.924795   Topic5  -5.1468  -0.0835
18   น้ำจิ้ม   0.995651   7.047878   Topic5  -5.2569   0.0424
88         2   0.890560   6.566591   Topic5  -5.3684   0.0016
171        3   0.865449   5.220549   Topic5  -5.3970   0.2024

[341 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
86        1  0.367872       
86        2  0.367872       
86        3  0.122624       
86        4  0.122624       
86        5  0.122624       
...     ...       ...    ...
47        4  0.116127  ไอติม
47        5  0.116127  ไอติม
48        5  0.742767      ​
582       3  0.970806      😆
583       3  0.970810      🤣

[406 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 1, 4, 5])

In [35]:
model.show_topic(4)

[('\u200b', 0.023045527),
 ('กิน', 0.020311752),
 ('อร่อย', 0.01933066),
 ('น้ำ', 0.01885596),
 ('ดี', 0.017987425),
 ('อาหาร', 0.016935272),
 ('ทาน', 0.016387703),
 ('เลือก', 0.014316569),
 ('ซุป', 0.013437248),
 ('บริการ', 0.012602834)]

In [36]:
df['topics'] = df['review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][0])
df['score'] = df['review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][1])

In [37]:
df[['Restaurant', 'cleansed_text', 'topics', 'score']]

,Restaurant,cleansed_text,topics,score
0,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,4,0.998319
1,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,รสชาติเหมือนทุกสาขา แต่สาขานี้ บริการดี ที่นั่...,4,0.997321
2,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ชาบูพรีเมียมสไตล์ญี่ปุ่น เนื้อดีมากก มาทานช่วง...,2,0.999246
3,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,เนื้อดี icecreamดี คุ้มค่าดี ถ้านึกถึงชาบูที่ม...,2,0.996420
4,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,อาหารมีคุณภาพ บริการดีค่ะ เดินมาหน้าร้านแล้วได...,1,0.997314
5,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,กินให้อิ่ม ที่ momo ร้านบุฟเฟ่ต์่ต์ ชาบูแนวญี่...,1,0.996423
6,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ชาบู สุกกี้สไลด์ญี่ปุ่นที่แท้ทรู number ...,0,0.998366
7,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ร้านชาบูแฟรนไชส์ ที่ราคาเหมาะสมกับคุณภาพ ถือว่...,0,0.997739
8,Shabushi (ชาบูชิ) เดอะมอลล์บางกะปิ ชั้น G,อร่อยคุ้ม ของทานเล่นเยอะมาก มา านที่ขาบูชิต้อง...,0,0.996738
9,Shabushi (ชาบูชิ) เดอะมอลล์บางกะปิ ชั้น G,ทานอีก ชอบกุ้งทอดเทมปุระ ใครชอบกุ้งทอดเทมปุระ ...,1,0.998045


In [38]:
pd.crosstab(df['Restaurant'], df['topics'])

topics,0,1,2,3,4
Restaurant,,,,,
Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,2,2,2,0,2
Shabushi (ชาบูชิ) เดอะมอลล์บางกะปิ ชั้น G,1,3,1,3,0
ข้าน้อยขอชาบู,2,2,0,0,1


In [ ]:
df.groupby('topics')['Rating'].describe()

,count,mean,std,min,25%,50%,75%,max
topics,,,,,,,,
0,5.0,4.0,0.707107,3.0,4.0,4.0,4.0,5.0
1,2.0,4.0,0.000000,4.0,4.0,4.0,4.0,4.0
2,10.0,4.0,0.666667,3.0,4.0,4.0,4.0,5.0
3,1.0,5.0,NaN,5.0,5.0,5.0,5.0,5.0
4,2.0,3.0,0.000000,3.0,3.0,3.0,3.0,3.0
